Nom, prenom et groupe

# TP6 : Maximum de vraisemblance via l'algorithme de montée de gradient

In [ ]:
# Chargement des bibliothèques
from matplotlib import pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs

print('Bibliothèques chargées')

## Problème A (à traiter avec l'enseignant) ##

On considère les données $X_j=(X_{j1},X_{j2})\mapsto Y_j$, pour $j\in\{1,2,3,4\}$, avec :

$X_1=(2,0)$, $X_2=(3,2)$, $X_3=(0,3)$. $X_3=(3,4)$, et

$Y_1=0$, $Y_2=0$, $Y_3=1$, $Y_4=1$.

Ces données sont représentées ci-dessous.

In [ ]:
# Définition des données

X1=[2,3,0,3]
X2=[0,2,3,4]
Y=[0,0,1,1]

#Représentation graphique des données

plt.axis([-1,5,-1,5]) #xmin,xmax,ymin,ymax
for i in range(len(X1)):
    if Y[i]==0:
        color='blue'
    else:
        color='red'
    plt.plot([X1[i]],[X2[i]],'o',c=color)
plt.show()

Dans la suite, on va chercher des coefficients $b$ et $W=\begin{pmatrix}w_1\\w_2\end{pmatrix}$ maximisant la vraisemblance de la loi de probabilité définie par $$\Pr(Y=1|X)=\sigma(b+w_1x_1+w_2x_2).$$
où $\sigma$ est la fonction sigmoïde, définie pour tout $x$ dans $R$ par $$\sigma(x)=\frac{1}{1+e^{-x}}.$$

La vraisemblance de cette loi est donnée par 
$$L(b,W)=\prod\limits_{j:Y_j=1}\sigma(b+w_1x_{j1}+w_2x_{j2})\times\prod\limits_{j:Y_j=0}(1-\sigma(b+w_1x_{j1}+w_2x_{j2}))$$

On rappelle (vu en cours) que $\sigma(x)\simeq 1$ dès que $x>>0$ ($x$ très supérieur à $0$), et $\sigma(x)\simeq 0$ dès que $x<<0$ ($x$ très inférieur à $0$).

La vraisemblance est donc grande si les coefficients $b$, $w_1$ et $w_2$ vérifient pour tout $j\in\{1,2,3,4\}$ :

$$(C_j):\left\{\begin{array}{l}b+w_1x_{j1}+w_2x_{j2}>>0\text{ si }Y_j=1\\b+w_1x_{j1}+w_2x_{j2}<<0\text{ si }Y_j=0\end{array}\right.$$


### Résolution "à la main"

**Exercice**

1. Ecrire les 4 conditions $(C_j)$ que doivent satisfaire $b$, $w_1$ et $w_2$ :
  - Condition (C1) issue de la donnée X1,Y1 : ...
  - Condition (C2) issue de la donnée X2,Y2 : ...
  - Condition (C3) issue de la donnée X3,Y3 : ...
  - Condition (C4) issue de la donnée X4,Y4 : ...

2. Proposer une valeur pour $b$, $w_1$, $w_2$ satisfaisant les 4 conditions précédentes.

*Indication* : tracer sur papier une droite séparant les données et un point $Z=(x_1(Z),x_2(Z))$ situé sur cette droite, puis déterminer les coefficients $b$, $w_1$ et $w_2$ d'une équation la forme $b+w_1x_1+w_2x_2=0$ de cette droite à partir des propriétés :
- le gradient $(w_1,w_2)$ de cette droite est orienté vers les $Y=1$
- les coordonnées de $Z$ satisfont l'équation de la droite : $b+w_1x_1(Z)+w_2x_2(Z)=0$

Réponse : b=.... , w1=.... , w2=....

3. Vérifier que la solution trouvée à la question précédente vérifie les 4 conditions de la question 1 :

  - Condition (C1) issue de la donnée X1,Y1 : ... : vérifiée
  - Condition (C2) issue de la donnée X2,Y2 : ... : vérifiée
  - Condition (C3) issue de la donnée X3,Y3 : ... : vérifiée
  - Condition (C4) issue de la donnée X4,Y4 : ... : vérifiée


### Visualisation de la solution

In [ ]:
# Définition de la fonction sigma
def sigma(x):
    return 1/(1+np.exp(-x))

print('Fonction sigma enregistrée')

**Exercice**

Compléter la cellule ci-dessous afin de compléter le graphique généré avec les courbes de niveaux de la fonctionn $(x_1,x_2)\mapsto\sigma(b+w_1x_1+w_2x_2)$ trouvée dans l'exercice précédent

In [ ]:
# Courbes de niveau de la fonction sigma(b+w1x1+w2x2)

# Calcul du tableau de valeurs (compléter avec les bonnes valeurs de b, w1, w2)
x1list=np.linspace(-1,5,50)
x2list=np.linspace(-1,5,50)
sigmalist=[[sigma(...+...*x1+...*x2) for x1 in x1list] for x2 in x2list]

# Dessin des courbes de niveau
plt.axis([-1,5,-1,5])
plt.contour(x1list,x2list,sigmalist,100) # dessine 100 courbes de niveau de sigma(b+w1x1+w2x2)
plt.colorbar()

# Nuage de points
for i in range(len(X1)):
    if Y[i]==0:
        color='blue'
    else:
        color='red'
    plt.plot([X1[i]],[X2[i]],'o',c=color)
plt.show()

### Détermination algorithmique d'une solution

Dans la suite, on va déterminer une fonction $b+W\cdot X$ séparant au mieux les données à l'aide d'un algorithme de *montée* de gradient sur la fonction de *log-vraisemblance*, qu'il s'agit de *maximiser*.

On rappelle que la log vraisemblance est donnée par :
$$\ln(L(b,W))=\sum\limits_{j:Y_j=1}\ln(\sigma(b+w_1x_{j1}+w_2x_{j2}))+\sum\limits_{j:Y_j=0}\ln((1-\sigma(b+w_1x_{j1}+w_2x_{j2})))$$

L'algorithme de montée de gradient est tel que, à chaque itération on calcule :
- $b[i+1]=b[i]+\tau\times\frac{\partial \ln(L(b,W))}{\partial b}$
- $w_1[i+1]=w_1[i]+\tau\times\frac{\partial \ln(L(b,W))}{\partial w_1}$
- $w_2[i+1]=w_2[i]+\tau\times\frac{\partial \ln(L(b,W))}{\partial w_2}$

Pour le programmer, on a donc besoin des dérivées de la log vraisemblance, données par (cf cours) :
- $\frac{\partial \ln(L(b,W))}{\partial b}=\sum\limits_j\left(Y_j-\sigma(b+w_1x_{j1}+w_2x_{j_2}\right)$
- $\frac{\partial \ln(L(b,W))}{\partial w_1}=\sum\limits_j\left(Y_j-\sigma(b+w_1x_{j1}+w_2x_{j_2}\right)x_{j1}$
- $\frac{\partial \ln(L(b,W)}{\partial w_2}=\sum\limits_j\left(Y_j-\sigma(b+w_1x_{j1}+w_2x_{j_2}\right)x_{j2}$


Les deux cellules ci-dessous définisse les fonctions :
- `gradlnL(b,w1,w2)`, qui renvoit le gradient de la log-vraisemblance calculée en (b,w1,w2)
- `montee(b,w1,w2,tau,tolerance,nbiterationsmax)`, qui effectue l'algorithme de montee de gradient et renvoit la liste `[b[i],w1[i],w2[i]]` des coefficients atteints par l'algorithme

In [ ]:
# Fonction renvoyant le gradient de la log vraisemblance associée à b,w1,w2
def gradlnL(b,w1,w2):
    return [np.sum([Y[j]-sigma(b+w1*X1[j]+w2*X1[j]) for j in range(len(X1))]),
            np.sum([(Y[j]-sigma(b+w1*X1[j]+w2*X2[j]))*X1[j] for j in range(len(X1))]),
            np.sum([(Y[j]-sigma(b+w1*X1[j]+w2*X2[j]))*X2[j] for j in range(len(X1))])
           ] 
print('Fonction gradlnL enregistrée')

In [ ]:
# Algorithme de montee de gradient initialisé
def montee(b,w1,w2,tau,tolerance,NbIterationsMax):
    diverge=False
    L=[]
    for i in range(NbIterationsMax):
        try: # traitement des erreurs si l'algorithme diverge
            if (b==float('inf')) or (w1==float("inf")) or (w2==float("inf")):
                raise(OverflowError)
            g = gradlnL(b,w1,w2)
            if g[0]**2+g[1]**2+g[2]**2<tolerance:
                print('L\'algorithme a convergé en',i,'itérations. \nSolution atteinte :\n b=',b,'\n w1=',w1,'\n w2=',w2,'\nGradient :',g,'\nNorme du gradient:',g[0]**2+g[1]**2+g[2]**2)
                L.append([b,w1,w2])
                return L
            L.append([b,w1,w2])
            b=b+tau*g[0]
            w1=w1+tau*g[1]
            w2=w2+tau*g[2]
        except OverflowError as err: # traitement de l'erreur "overflow"
            print('L\'algorithme a divergé.\n Solution atteinte :\n b=',b,'\n w1=',w1,'\n w2=',w2,'\nGradient :',g,'\nNorme du gradient:',g[0]**2+g[1]**2+g[2]**2)
            diverge=True
            break
    if (diverge==False):        
        print('L\'algorithme n\'a pas convergé.\n Solution atteinte :\n b=',b,'\n w1=',w1,'\n w2=',w2,'\nGradient :',g,'\nNorme du gradient:',g[0]**2+g[1]**2+g[2]**2)
    return L

print('Fonction montee enregistrée')

**Exercice**

La cellule ci-dessous permet d'effectuer l'algorithme, puis de représenter les droites d'équation `b[i]+w1[i]*x1+w2[i]*x2=0` atteintes par l'algorithme. L'algorithme converge si le gradient de la log-vraisemblance atteint est suffisamment petit (inférieur à `tolerance`).

1. Effectuer l'algorithme initialisé à la droite d'équation $x_2=1$ (déterminer les $b$, $w_1$ et $w_2$ associés), avec un pas $\tau=0.1$, une tolérance de $0.1$ et un nombre d'itérations maximum de 50 (vous devriez obtenir une convergence en 27 itérations).
2. En utilisant la sortie texte de l'lagorithme, donner l'équation de la droite atteinte par l'algorithme. Cette droite sépare-t-elle linéairement les données ?


In [ ]:
# Calculs de la suite des [b[i],w1[i],w2[i]] via l'algorithme
ListeBW=montee(...,...,...,0.01,0.1,50)

# Représentation graphique des droites b[i]+w1[i]*x1+w2[i]*x2
for i in range(len(ListeBW)):
    W=ListeBW[i]
    b=W[0]
    w1=W[1]
    w2=W[2]
    if (w2!=0):
        plt.plot([-1,5],[(-w1*(-1)-b)/w2,(-w1*5-b)/w2],c="black",alpha=0.1+0.9*i/(len(ListeBW)-1))
    elif (w1!=0):
        plt.plot([(-w2*(-1)-b)/w1,(-w2*5-b)/w1],[-1,5],c="black",alpha=0.1+0.9*i/(len(ListeBW)-1))        
    # alpha est un paramètre de transparence variant ici de 0.1 à 0.9 avec la valeur de i

# Nuage de points
for i in range(len(X1)):
    if Y[i]==0:
        color='blue'
    else:
        color='red'
    plt.plot([X1[i]],[X2[i]],'o',c=color)
plt.show()

# Problème B (à traiter en autonomie) #

À vous de généraliser les calculs précédents avec un nuage de 40 données

In [ ]:
# Création aléatoire des données

# Cette cellule n'est à exécuter qu'une fois, afin de générer des données aléatoires,
# et afin de les recopier dans la cellule suivante

# on génère 40 données (X1j,X2j,Yj), avec Yj=0 ou 1
# données centrées autour de 2 centres
X, Y = make_blobs(n_samples=40, centers=[[-0.5,-0.5],[0.5,0.5]], n_features=2, cluster_std=0.5,center_box=(-1,1))
print('Liste X1 des Xj1 (à copier coller dans la cellule ci-dessous): \n\n',np.array2string(X[:,0],separator=',',max_line_width=10000).replace('\n', ''),'\n')
print('Liste X2 des Xj2 (à copier coller dans la cellule ci-dessous): \n\n',np.array2string(X[:,1],separator=',',max_line_width=10000).replace('\n', ''),'\n')
print('\nListe Y des Yj (à copier coller dans la cellule ci-dessous): \n\n',np.array2string(Y,separator=',',max_line_width=10000).replace('\n', ''),'\n')

In [ ]:
# Modifier en copiant collant avec les données générées aléatoirement ci-dessus
# attention à bien identifier les crochets fermant et ouvrant

X1=[...]
X2=[...]
Y=[...]

In [ ]:
# Visualisation du nuage de points
# (Attention à la fenêtre, qui est différente de celle du problème A)

...

**Exercice**

1. Utiliser la cellule ci-dessous pour déterminer une droite séparant au mieux les données à l'aide de l'algorithme de montée de gradient (algorithme convergent avec avec une tolerance à 0.1, 50 itérations max)
2. Utiliser la cellule suivante pour représenter les étapes de l'algorithme ainsi que le nuage de points.
3. Compléter la cellule suivante avec les paramètres utilisés et avec les résultats obtenus (pour une loi $(x1,x2)\mapsto \sigma(b+w_1x_1+w_2x_2)$ obtenue, un *faux positif* est tel que $\sigma(b+w_1x_{j1}+w_2x_{j2})>0.5$ alors que $Y_j=0$, et un *faux négatif* est tel que $\sigma(b+w_1x_{j1}+w_2x_{j2})<0.5$ alors que $Y_j=1$).

In [ ]:
# Solution algorithmique


In [ ]:
# Représentation graphique


**Réponses à compléter **

- Valeurs initiales : ...

- tolerance : ..., facteur $\tau$ : ...
- Convergence en ... itérations

- Droite obtenu : ...

- Nombre de faux positifs :

- Nombre de faux négatifs :